In [1]:
import os
import datetime
import torch
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import wandb
import sys
import matplotlib.pyplot as plt
import pickle

# from RMDT_model.RMDT import mem_transformer_v2

import os
import sys
sys.path.append("../../")

from VizDoom.VizDoom_src.get_vizdoom_dataset import get_dataset


libgomp: Invalid value for environment variable OMP_NUM_THREADS

libgomp: Invalid value for environment variable OMP_NUM_THREADS


# Load VizDoom-Two-Colors data

In [2]:
# train_pickle_file = 'VizDoom/VizDoom_data/train_VizDoom_Two_Colors_Column_disappear_delay_45_no_walls_agent_p1_01.pickle'
val_pickle_file = '../../VizDoom/VizDoom_data/val_VizDoom_Two_Colors_Column_disappear_delay_45_no_walls_agent_p1_01.pickle'

# DATA2_train = []
DATA2_val = []

# with open(train_pickle_file, 'rb') as f:
#     while True:
#         try:
#             data = pickle.load(f)
#             DATA2_train.append(data)
#         except EOFError:
#             break
            
with open(val_pickle_file, 'rb') as f:
    while True:
        try:
            data = pickle.load(f)
            DATA2_val.append(data)
        except EOFError:
            break
            
# DATA_train = {}
DATA_val = {}

# for key in tqdm(DATA2_train[0].keys()):
#     DATA_train[key] = [d[key] for d in DATA2_train]
    
for key in tqdm(DATA2_val[0].keys()):
    DATA_val[key] = [d[key] for d in DATA2_val]

UnpicklingError: pickle data was truncated

In [10]:
print(DATA_val.keys())
# print(len(DATA_train['obs']))
print(len(DATA_val['obs']))

dict_keys(['obs', 'action', 'reward', 'done', 'is_red'])
1024
256


In [21]:
a = 0
b = 1
if a == 0 \
    and b == 1:
    print("rofl")

rofl
